In [1]:
import sys
import glob
import math
import agama
import numpy as np
from astropy import units as u
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from pygaia.astrometry.coordinates import CoordinateTransformation, Transformations
#from pygaia.astrometry.vectorastrometry import cartesianToSpherical, astrometryToPhaseSpace,phaseSpaceToAstrometry
from pygaia.astrometry.vectorastrometry import astrometry_to_phase_space

In [2]:
import pandas as pd

et the units

In [3]:
agama.setUnits(mass=1,length=1,velocity=1)

arameters for debugging<br>
nput_file_name = '../data/AEGIS/Original/gaia/aegis_rv/Trimmed_original_data_inputs_rv_aegis_gaia_dist_1.csv'<br>
otential = agama.Potential("../potentials/McMillan17.ini")

et the input file name

In [4]:
# input_file_name = glob.glob('temporary*')[0]

et the output file name

In [5]:
output_file_name = "star_data/filtered_stars_outputs.csv"

et the potential and set the action finder

In [6]:
potential = agama.Potential("star_data/McMillan17.ini")
act_finder = agama.ActionFinder(potential)

In [7]:
def ICRS_to_GAL(phi, theta, muphistar, mutheta):
    """
        phi       - The longitude-like angle of the position of the source (radians)
        theta     - The latitude-like angle of the position of the source (radians)
        muphistar - Value of the proper motion in the longitude-like angle, multiplied by cos(latitude).
        mutheta   - Value of the proper motion in the latitude-like angle.
        """
       # transformation ICRS to GAL
    ctICRS2GAL =CoordinateTransformation(Transformations.ICRS2GAL)#
    ell, bee = ctICRS2GAL.transform_sky_coordinates(phi,theta)
    muellstar, mubee = ctICRS2GAL.transform_proper_motions(phi,theta, muphistar,mutheta)
    return ell,bee,muellstar,mubee

In [8]:
def astrometric_2_cartesian(DM, ell_radian, bee_radian, HRV, muellstar, mubee):
    parallax_mas = np.power(10., 2.-DM/5.)
    xHelio_pc, yHelio_pc, zHelio_pc, vxHelio, vyHelio, vzHelio = astrometry_to_phase_space(ell_radian, bee_radian, parallax_mas, muellstar, mubee, HRV)
    xHelio_kpc = xHelio_pc/1000.
    yHelio_kpc = yHelio_pc/1000.
    zHelio_kpc = zHelio_pc/1000.
    return xHelio_kpc, yHelio_kpc, zHelio_kpc, vxHelio, vyHelio, vzHelio

In [9]:
def cartesian_2_cylindrical(x,y,z,vx,vy,vz):
    R =np.sqrt(x**2 + y**2)
    r =np.sqrt(x**2 + y**2 + z**2)
    phi=np.arctan2(y,x)
    vR     = (x*vx + y*vy)/R
    vTHETA = (R*vR - z*vz)/r
    vPHI   = (x*vy - y*vx)/R
    return R, phi, vR, vTHETA, vPHI

alculate orbital parameters

In [10]:
def calculate_orbital_parameters(xGC_norm, yGC_norm, zGC_norm, vxGC_norm, vyGC_norm, vzGC_norm, inttime, numsteps):
    # galpy convention
    x_agama  =   xGC_norm
    y_agama  =   yGC_norm
    z_agama  =   zGC_norm
    vx_agama =   vxGC_norm
    vy_agama =   vyGC_norm
    vz_agama =   vzGC_norm
    #R_agama   =  np.sqrt(x_agama**2 + y_agama**2)
    #vR_agama  =  ( x_agama*vx_agama + y_agama*vy_agama )/R_agama
    #vT_agama  = -( x_agama*vy_agama - y_agama*vx_agama )/R_agama
    #phi_agama =  np.arctan2(y_agama,x_agama)
    #inttime=100.
    #numsteps=3000
    times = np.linspace(0, inttime, numsteps)
    times_c, c_orb_car = agama.orbit(ic=[x_agama,y_agama,z_agama,vx_agama,vy_agama,vz_agama], potential=potential, time=inttime, trajsize=numsteps)
    x = c_orb_car[:,0]
    y = c_orb_car[:,1]
    z = c_orb_car[:,2]
    vx= c_orb_car[:,3]
    vy= c_orb_car[:,4]
    vz= c_orb_car[:,5]
    R = np.sqrt(x**2 + y**2)
    r = np.sqrt(x**2 + y**2 + z**2)
    rmin = np.min(r)
    rmax = np.max(r)
    zmax = np.max(np.fabs(z))
    ecc  = (rmax-rmin)/(rmax+rmin)
    return rmin, rmax, zmax, ecc

osition and velocity of the Sun

In [15]:
_xGC_sun_kpc = -8.249
_yGC_sun_kpc =  0.
_zGC_sun_kpc =  0.
_vxGC_sun    =  11.1
_vyGC_sun    =  250.70
_vzGC_sun    =  7.25

pen the data file

In [16]:
data = []
with open("star_data/filtered_stars2.csv",'r') as file:
    data = file.readlines()

btain the RA, DEC, distance, radial velocity, pmRA, and pmDEC from the file<br>
ame list contains the stars which can be run through AGAMA while name_starred list are the stars<br>
hich have a '*' in the data indicating that they cannot have orbits calculated due to missing information<br>
he name_all list is a list of all the stars that are found in the input file

In [17]:
filtered_stars_df = pd.read_csv("star_data/filtered_stars2.csv")

In [18]:
filtered_stars_df.count()

Unnamed: 0                   552
source_id                    552
ra                           552
ra_error                     552
dec                          552
dec_error                    552
pmra                         552
pmra_error                   552
pmdec                        552
pmdec_error                  552
pmra_pmdec_corr              552
parallax                     552
parallax_error               552
phot_g_mean_mag              552
bp_rp                        552
dr2_radial_velocity          552
dr2_radial_velocity_error    552
U                            552
V                            552
W                            552
T                            552
Prograde                     552
V-232                        552
dtype: int64

In [19]:
len(filtered_stars_df.index[filtered_stars_df['parallax'] < 0])

0

In [16]:
filtered_stars_df = filtered_stars_df.drop(filtered_stars_df.index[filtered_stars_df['parallax'] < 0].tolist()[0])

IndexError: list index out of range

In [17]:
# filtered_stars_df.where(filtered_stars_df["source_id"]==4.890626968730639e+18, inplace=True)

In [20]:
filtered_stars_df = filtered_stars_df.reset_index()

In [23]:
# del filtered_stars_df["index"]
del filtered_stars_df["Unnamed: 0"]

In [24]:
filtered_stars_df

,source_id,ra,ra_error,dec,dec_error,pmra,pmra_error,pmdec,pmdec_error,pmra_pmdec_corr,...,phot_g_mean_mag,bp_rp,dr2_radial_velocity,dr2_radial_velocity_error,U,V,W,T,Prograde,V-232
0,3.189432e+18,63.032343,0.013695,-12.118837,0.012714,-6.449587,0.016815,-10.750589,0.016687,0.018852,...,11.260073,0.771560,4.687665,0.725956,-14.527353,8.499058,-4.568251,15.228687,1,223.500942
1,3.189440e+18,63.147321,0.038085,-12.053509,0.030295,17.961836,0.044886,-6.614329,0.038475,-0.110601,...,11.878635,1.048296,21.304344,0.654505,7.911635,-8.413707,1.950579,8.148541,0,240.413707
2,2.471977e+18,20.144456,0.023063,-8.872362,0.017459,75.216503,0.031480,-15.498916,0.019502,0.499377,...,11.587500,0.851440,14.657644,1.197960,44.409299,-36.835483,-3.925566,44.582462,0,268.835483
3,2.471978e+18,20.156585,0.019501,-8.827725,0.013724,15.430254,0.027197,-9.424502,0.016023,0.531574,...,11.687765,1.141704,7.125475,0.996812,21.559445,-48.303929,-4.914120,22.112400,0,280.303929
4,4.890596e+18,62.598973,0.006453,-25.771897,0.009285,12.649115,0.008598,13.180861,0.012657,-0.453528,...,9.783724,0.666553,29.874018,0.575682,16.442726,-0.981120,-7.898591,18.241463,0,232.981120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,1.833590e+18,302.685519,0.010022,23.999862,0.010915,-0.372788,0.011480,-2.403061,0.012906,-0.029631,...,12.936925,2.057656,18.921799,0.470167,-43.639656,15.422443,-8.711134,44.500600,1,216.577557
548,1.833590e+18,302.708357,0.009061,24.016987,0.009434,-3.497907,0.010459,-6.716860,0.011292,0.025704,...,13.568626,2.416790,23.487917,1.332360,-154.924803,-35.478746,-9.726123,155.229804,0,267.478746
549,1.833590e+18,302.738669,0.009251,24.014045,0.010056,-1.899784,0.010637,-5.130046,0.011925,-0.072106,...,13.737948,2.628884,-3.933907,1.064926,-177.727346,-80.383723,-35.807531,181.298618,0,312.383723
550,1.833590e+18,302.711956,0.014754,24.045495,0.015232,-2.916555,0.017104,-5.268033,0.018684,-0.127595,...,12.831676,3.184610,-16.689116,0.600616,-125.610253,-64.969656,-1.336542,125.617364,0,296.969656


In [25]:
name = []
name_all = []
name_starred = []
ra_dec = []
dist = []
dist_error = []
rv = []
rv_error = []
pmra = []
pmra_error = []
pmdec = []
pmdec_error = []
pmra_pmdec_corr_error = []
counter = -1
index_offset = 1



for i in range(filtered_stars_df.count()[0]):
    counter = counter + 1
    if counter != 0:
        split_line = filtered_stars_df.loc[i].to_string().split(' ')
        # print(filtered_stars_df.loc[i].to_string())
        name_all.append(filtered_stars_df.loc[i, "source_id"])
        if "*" in split_line:
            name_starred.append(filtered_stars_df.loc[i, "source_id"])
        else:
            name.append(filtered_stars_df.loc[i, "source_id"])
            ra_dec.append(str(filtered_stars_df.loc[i, "ra"]) + ' ' + str(filtered_stars_df.loc[i, "dec"]))
            if 1/float(filtered_stars_df.loc[i, "parallax"]) < 0:
                print(str(filtered_stars_df.loc[i, "source_id"]) + ' has a distance less than zero!' + str(filtered_stars_df.loc[i, "parallax"]))
            dist.append(1/float(filtered_stars_df.loc[i, "parallax"]))
            dist_error.append(filtered_stars_df.loc[i, "parallax_error"]/filtered_stars_df.loc[i, "parallax"]**2)
            
            rv.append(float(filtered_stars_df.loc[i, "dr2_radial_velocity"]))
            rv_error.append(float(filtered_stars_df.loc[i, "dr2_radial_velocity_error"]))
            pmra.append(float(filtered_stars_df.loc[i, "pmra"]))
            pmra_error.append(float(filtered_stars_df.loc[i, "pmra_error"])) 
            pmdec.append(float(filtered_stars_df.loc[i, "pmdec"])) 
            pmdec_error.append(float(filtered_stars_df.loc[i, "pmdec_error"]))
            pmra_pmdec_corr_error.append(float(filtered_stars_df.loc[i, "pmra_pmdec_corr"])) #

In [26]:
#4.890626968730639e+18

rint name, ra_dec, dist, dist_error, rv, rv_error, pmra, pmra_error, pmdec, pmdec_error, pmra_pmdec_corr_error

ut the coordinates into a bin to get them easier in degrees and radians if there are any

In [27]:
ra_rad = []
dec_rad = []
if name:
    coordinates = SkyCoord(ra_dec,unit=(u.hourangle,u.deg))
    for coord in coordinates:
        ra_rad.append(coord.ra.rad)
        dec_rad.append(coord.dec.rad)

#############################################################################<br>
alculate the actions and velocities and their errors using monte carlo methods

et the length and of the simulation

In [28]:
num_stars = len(name)
_N_MonteCarlo = 10
np.random.seed()  #123 for debug purposes

In [29]:
num_stars

551

rrays to put the results of the montecarlo simulation into

In [30]:
JrJzJphiE_Nstar_Nmontecarlo = np.ones((4, num_stars, _N_MonteCarlo))
XYZVxVy_Nstar_Nmontecarlo = np.ones((5, num_stars, _N_MonteCarlo))
vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo = np.ones((8, num_stars, _N_MonteCarlo))

In [31]:
print("hello")

hello


In [32]:
Jr_mean = []
Jr_std = []
Jz_mean = []
Jz_std = []
Jphi_mean = []
Jphi_std = []
Jx_mean = []
Jx_std = []
Jy_mean = []
Jy_std = []
vr_mean = []
vr_std = []
vx_mean = []
vx_std = []
vy_mean = []
vy_std = []
vz_mean = []
vz_std = []
vphi_mean = []
vphi_std = []
Lx_mean = []
Lx_std = []
Ly_mean = []
Ly_std = []
energy_mean = []
energy_std = []
phi_mean = []
phi_std = []
rperi_mean = []
rperi_std = []
rapo_mean = []
rapo_std = []
zmax_mean = []
zmax_std = []
ecc_mean = []
ecc_std = []
count = 0
for i in np.arange(0,num_stars):
    for j in np.arange(0,_N_MonteCarlo):
        RA_rad_ij = ra_rad[i] #no error
        DEC_rad_ij = dec_rad[i] #no error
        HRV_kms_ij = rv[i] + np.random.randn()*rv_error[i] #put in random error
        mean_pmRA_pmDEC = [pmra[i],pmdec[i]]
        #Get the error for proper motion RA and DEC
        e_pmRA = pmra_error[i]
        e_pmDEC = pmdec_error[i]
        e_pmRA_pmDEC = pmra_pmdec_corr_error[i]
        #Calculate the sigma from proper motion RA and DEC
        Sigma_pmRA = e_pmRA**2
        Sigma_pmDEC = e_pmDEC**2
        Sigma_pmRA_pmDEC = e_pmRA_pmDEC*e_pmRA*e_pmDEC
        Sigma = [[Sigma_pmRA, Sigma_pmRA_pmDEC],
            [Sigma_pmRA_pmDEC, Sigma_pmDEC]]
        #Create an array of the proper motion RA and DEC
        pmRAstar_masyr_ij_array, pmDECstar_masyr_ij_array = np.random.multivariate_normal(mean=mean_pmRA_pmDEC, cov=Sigma, size=1).T
        pmRAstar_masyr_ij = pmRAstar_masyr_ij_array[0]
        pmDECstar_masyr_ij = pmDECstar_masyr_ij_array[0]
        #Create a distance with error
        dist_kpc_ij = dist[i] + np.random.rand()*dist_error[i]
        
        while dist_kpc_ij < 0.0:
            dist_kpc_ij = dist[i] + np.random.rand()*dist_error[i]
        
        parallax_mas_ij = 1.0/dist_kpc_ij
        #Calculate a magnitude from the calculated parallax
        DM_mag_ij = 5.*(np.log10(1./parallax_mas_ij) + 2.0)
        #Put the star into galactic coordinates
        l_rad_ij, b_rad_ij, pmlstar_masyr_ij, pmbstar_masyr_ij = ICRS_to_GAL(RA_rad_ij, DEC_rad_ij, pmRAstar_masyr_ij, pmDECstar_masyr_ij)
        #Convert to Cartesian
        xH, yH, zH, vxH, vyH, vzH = astrometric_2_cartesian(DM_mag_ij, l_rad_ij, b_rad_ij, HRV_kms_ij, pmlstar_masyr_ij, pmbstar_masyr_ij)
        #Normalize the position
        xGC_norm = (xH + _xGC_sun_kpc)
        yGC_norm = (yH + _yGC_sun_kpc)
        zGC_norm = (zH + _zGC_sun_kpc)
        #normalize  velocity
        vxGC_norm = (vxH + _vxGC_sun)
        vyGC_norm = (vyH + _vyGC_sun)
        vzGC_norm = (vzH + _vzGC_sun)
        #cylindrical coordinate
        R, phi, vR, vTHETA, vPHI = cartesian_2_cylindrical(xGC_norm, yGC_norm, zGC_norm, vxGC_norm, vyGC_norm, vzGC_norm)
        #vPERP = np.sqrt(vR**2 + vzGC_norm**2)
        #Calculate the energy
        energy = potential.potential([xGC_norm,yGC_norm,zGC_norm]) + 0.5*(vxGC_norm**2+vyGC_norm**2+vzGC_norm**2)
        #Calculate the actions
        xv6d = np.array( [xGC_norm, yGC_norm, zGC_norm, vxGC_norm, vyGC_norm, vzGC_norm]).T
        Jr, Jz, Jphi = act_finder(xv6d)
        #Since we are using a right-handed coordinate system
        #The phi results need to have sign switch in order
        #for prograde stars to have a positive action
        vPHI = -vPHI
        # print(Jr)
        rmin, rmax, zmax, ecc= np.nan, np.nan, np.nan, np.nan
        if (np.isnan(Jr)):
                Jr = -1.
        if (np.isnan(Jz)):
            Jz = -1.
        if (np.isnan(energy)):
            energy = 1
    #Get the orbital parameters
        if (energy<0.):
            rmin, rmax, zmax, ecc = calculate_orbital_parameters(xGC_norm,yGC_norm,zGC_norm,vxGC_norm,vyGC_norm,vzGC_norm,100,10001)
        else:
            Jr, Jz, Jphi, energy, vR, vzGC_norm, vPHI, phi = np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
        # print(i, j)
        #Put the actions and the energy into an array
        JrJzJphiE_Nstar_Nmontecarlo[0,i,j] = Jr
        JrJzJphiE_Nstar_Nmontecarlo[1,i,j] = Jz
        JrJzJphiE_Nstar_Nmontecarlo[2,i,j] = Jphi
        JrJzJphiE_Nstar_Nmontecarlo[3,i,j] = energy
        #Put the velocities into an array
        vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[0,i,j] = vR
        vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[1,i,j] = vzGC_norm
        vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[2,i,j] = vPHI
        vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[3,i,j] = phi
        #Put the positions and remaining velocities into an array
        XYZVxVy_Nstar_Nmontecarlo[0,i,j] = xGC_norm
        XYZVxVy_Nstar_Nmontecarlo[1,i,j] = yGC_norm
        XYZVxVy_Nstar_Nmontecarlo[2,i,j] = zGC_norm
        XYZVxVy_Nstar_Nmontecarlo[3,i,j] = vxGC_norm
        XYZVxVy_Nstar_Nmontecarlo[4,i,j] = vyGC_norm
        #Put the distances and eccentricities into an array
        vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[4,i,j] = rmin
        vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[5,i,j] = rmax
        vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[6,i,j] = zmax
        vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[7,i,j] = ecc
        #Flush the output to the output file
        sys.stdout.flush()

        #Calculate the mean and std of the actions
    count+=1
    if not count%10:
        print(count)

    Jr_mean.append(np.nanmean(JrJzJphiE_Nstar_Nmontecarlo[0,i]))
    Jr_std.append(np.nanstd(JrJzJphiE_Nstar_Nmontecarlo[0,i]))
    Jz_mean.append(np.nanmean(JrJzJphiE_Nstar_Nmontecarlo[1,i]))
    Jz_std.append(np.nanstd(JrJzJphiE_Nstar_Nmontecarlo[1,i]))
    Jphi_mean.append(np.nanmean(JrJzJphiE_Nstar_Nmontecarlo[2,i]))
    Jphi_std.append(np.nanstd(JrJzJphiE_Nstar_Nmontecarlo[2,i]))
        #Calculate the mean and std of the energy
    energy_mean.append(np.nanmean(JrJzJphiE_Nstar_Nmontecarlo[3,i]))
    energy_std.append(np.nanstd(JrJzJphiE_Nstar_Nmontecarlo[3,i]))
    #Calculate the mean and std of the velocities
    vr_mean.append(np.nanmean(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[0,i]))
    vr_std.append(np.nanstd(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[0,i]))
    vz_mean.append(np.nanmean(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[1,i]))
    vz_std.append(np.nanstd(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[1,i]))
    vphi_mean.append(np.nanmean(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[2,i]))
    vphi_std.append(np.nanstd(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[2,i]))
    #Calculate the mean and std of the angular momenta
    Lx_mean.append(np.nanmean(XYZVxVy_Nstar_Nmontecarlo[1,i]*vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[1,i] - XYZVxVy_Nstar_Nmontecarlo[2,i]*XYZVxVy_Nstar_Nmontecarlo[4,i]))
    Lx_std.append(np.nanstd(XYZVxVy_Nstar_Nmontecarlo[1,i]*vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[1,i] - XYZVxVy_Nstar_Nmontecarlo[2,i]*XYZVxVy_Nstar_Nmontecarlo[4,i]))
    Ly_mean.append(np.nanmean(XYZVxVy_Nstar_Nmontecarlo[2,i]*XYZVxVy_Nstar_Nmontecarlo[3,i] - XYZVxVy_Nstar_Nmontecarlo[0,i]*vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[1,i]))
    Ly_std.append(np.nanstd(XYZVxVy_Nstar_Nmontecarlo[2,i]*XYZVxVy_Nstar_Nmontecarlo[3,i] - XYZVxVy_Nstar_Nmontecarlo[0,i]*vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[1,i]))
    #Calculate the mean and std of the distances and eccentricities
    rperi_mean.append(np.nanmean(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[4,i]))
    rperi_std.append(np.nanstd(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[4,i]))
    rapo_mean.append(np.nanmean(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[5,i]))
    rapo_std.append(np.nanstd(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[5,i]))
    zmax_mean.append(np.nanmean(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[6,i]))
    zmax_std.append(np.nanstd(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[6,i]))
    ecc_mean.append(np.nanmean(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[7,i]))
    ecc_std.append(np.nanstd(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[7,i]))
    #Calculate the mean and std of phi (Used to convert to Cartesian Coordinates)
    phi_mean.append(np.nanmean(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[3,i]))
    phi_std.append(np.nanstd(vr_vz_vphi_phi_rPERI_rAPO_zMAX_ecc__Nstar__NMonteCarlo[3,i]))
    #Calculate the mean and std of Cartesian actions and velocities
    Jx_mean.append(np.nanmean(Jr_mean[i]*np.cos(phi_mean[i])))
    Jx_std.append(np.sqrt(np.cos(phi_mean[i])**2*Jr_std[i]*2+Jr_mean[i]**2*np.sin(phi_mean[i])**2*phi_std[i]**2))
    Jy_mean.append(np.nanmean(Jr_mean[i]*np.sin(phi_mean[i])))
    Jy_std.append(np.sqrt(np.sin(phi_mean[i])**2*Jr_std[i]**2+Jr_mean[i]**2*np.cos(phi_mean[i])**2*phi_std[i]**2))
    vx_mean.append(np.nanmean(XYZVxVy_Nstar_Nmontecarlo[3,i]))
    vx_std.append(np.nanstd(XYZVxVy_Nstar_Nmontecarlo[3,i]))
    vy_mean.append(np.nanmean(XYZVxVy_Nstar_Nmontecarlo[4,i]))
    vy_std.append(np.nanstd(XYZVxVy_Nstar_Nmontecarlo[4,i]))


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520


/var/folders/nz/mn73d3yd7l79gqp918nt54480000gn/T/ipykernel_59435/3536347129.py:134: RuntimeWarning: Mean of empty slice
  Jr_mean.append(np.nanmean(JrJzJphiE_Nstar_Nmontecarlo[0,i]))
/Users/anuragsah00/anaconda3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/var/folders/nz/mn73d3yd7l79gqp918nt54480000gn/T/ipykernel_59435/3536347129.py:136: RuntimeWarning: Mean of empty slice
  Jz_mean.append(np.nanmean(JrJzJphiE_Nstar_Nmontecarlo[1,i]))
/var/folders/nz/mn73d3yd7l79gqp918nt54480000gn/T/ipykernel_59435/3536347129.py:138: RuntimeWarning: Mean of empty slice
  Jphi_mean.append(np.nanmean(JrJzJphiE_Nstar_Nmontecarlo[2,i]))
/var/folders/nz/mn73d3yd7l79gqp918nt54480000gn/T/ipykernel_59435/3536347129.py:141: RuntimeWarning: Mean of empty slice
  energy_mean.append(np.nanmean(JrJzJphiE_Nstar_Nmontecarlo[3,i]))
/var/folders/nz/mn73d3yd7l79gqp918nt54480000gn/T/ipykernel_

530
540
550


In [33]:
len(vr_std)

551

ut the velocities, actions, energy, Apocentric/Pericentric Distance, z max and the eccentricity into output file

In [32]:
with open(output_file_name,'w') as file:
    file.write('Name,v_r(km/s),v_r_std(km/s),v_phi(km/s),v_phi_std(km/s),v_x(km/s),v_x_std(km/s),v_y(km/s),v_y_std(km/s),v_z(km/s),v_z_std(km/s),j_r(kpc km s^-1),j_r_std(kpc km s^-1),j_phi(kpc s^(-1)),j_phi_std(kpc km s^(-1)),j_x(kpc km s^(-1)),j_x_std(kpc km s^(-1)),j_y(kpc km s^(-1)),j_y_std(kpc km s^(-1)),j_z(kpc km s^(-1)),j_z_std(kpc km s^(-1)),L_x(km s^(-1) kpc),L_x_std(km s^(-1) kpc),L_y(km s^(-1) kpc),L_y_std(km s^(-1) kpc),energy_mean(km^2 s^(-2)),energy_std(km^2 s^(-2)),rperi_mean(kpc),rperi_std(kpc),rapo_mean(kpc),rapo_std(kpc),zmax_mean(kpc),zmax_std(kpc),ecc_mean,ecc_std\n')
    for i in np.arange(0, 307):
        #Check to see if AGAMA ran for the star
        if name_all[i] in name and not name_all[i] in name_starred:
            #Get the index of the star in the good name list
            
            name_index = name.index(name_all[i])
            #Write the name of the object
            file.write(str(name[name_index])+',')
            # print(")
            #Write the velocities
            file.write(str(vr_mean[name_index])+',')
            file.write(str(vr_std[name_index])+',')
            file.write(str(vphi_mean[name_index])+',')
            file.write(str(vphi_std[name_index])+',')
            file.write(str(vx_mean[name_index])+',')
            file.write(str(vx_std[name_index])+',')
            file.write(str(vy_mean[name_index])+',')
            file.write(str(vy_std[name_index])+',')
            file.write(str(vz_mean[name_index])+',')
            file.write(str(vz_std[name_index])+',')
            #Write the actions
            file.write(str(Jr_mean[name_index])+',')
            file.write(str(Jr_std[name_index])+',')
            file.write(str(Jphi_mean[name_index])+',')
            file.write(str(Jphi_std[name_index])+',')
            file.write(str(Jx_mean[name_index])+',')
            file.write(str(Jx_std[name_index])+',')
            file.write(str(Jy_mean[name_index])+',')
            file.write(str(Jy_std[name_index])+',')
            file.write(str(Jz_mean[name_index])+',')
            file.write(str(Jz_std[name_index])+',')
            #Write the angular momenta
            file.write(str(Lx_mean[name_index])+',')
            file.write(str(Lx_std[name_index])+',')
            file.write(str(Ly_mean[name_index])+',')
            file.write(str(Ly_std[name_index])+',')
            #Write the energy
            file.write(str(energy_mean[name_index])+',')
            file.write(str(energy_std[name_index])+',')
            #Write the Apocentric/Pericentric Distance, z max and the eccentricity
            file.write(str(rperi_mean[name_index])+',')
            file.write(str(rperi_std[name_index])+',')
            file.write(str(rapo_mean[name_index])+',')
            file.write(str(rapo_std[name_index])+',')
            file.write(str(zmax_mean[name_index])+',')
            file.write(str(zmax_std[name_index])+',')
            file.write(str(ecc_mean[name_index])+',')
            file.write(str(ecc_std[name_index])+'\n')
        #Check to see if there were no parameters for AGAMA to run
        elif name_all[i] in name_starred and not name_all[i] in name:
            file.write(name_all[i]+',*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*\n')
        #Make sure nothing went wrong
        else:
            file.write(name_all[i]+',SOMETHING WENT WRONG! STAR NOT FOUND!')